In [35]:
import pandas as pd
from xml.dom import minidom
import numpy as np
from unidecode import unidecode
sys.path.insert(1, '../utils')
from read_config import read_config_file
from nltk.tokenize import RegexpTokenizer

In [2]:
inputs_path = '../inputs/'
results_path = '../results/'

## Read Config File

In [5]:
config_dict = {}
read_config_file(inputs_path + 'GLI.CFG', config_dict)
config_dict

{'LEIA': ['cf74.xml',
  'cf75.xml',
  'cf76.xml',
  'cf77.xml',
  'cf78.xml',
  'cf79.xml'],
 'ESCREVA': ['lista.csv']}

## Parse XML

In [54]:
records_data = []
for xml_file in  config_dict['LEIA']:
    parsed_xml = minidom.parse(inputs_path + xml_file)
    records = parsed_xml.getElementsByTagName('RECORD')
    
    for record in records:
        recordnum = record.getElementsByTagName('RECORDNUM')[0].firstChild.nodeValue.strip()
        abstract = record.getElementsByTagName('ABSTRACT')
        extract = record.getElementsByTagName('EXTRACT')
        
        if len(abstract) > 0:
            text = unidecode(abstract[0].firstChild.nodeValue.strip().lower())
        elif len(extract) > 0:
            text = unidecode(extract[0].firstChild.nodeValue.strip().lower())
        else:
            text = ''
            
        records_data.append({'RECORDNUM': recordnum, 'TEXT': text})

In [55]:
records_df = pd.DataFrame(records_data)

## Generate List

In [56]:
gli_dict = {}
for index, row in records_df.iterrows():
    text = row['TEXT']
    recordnum = row['RECORDNUM']
    tokenizer = RegexpTokenizer(r'\w+')
    words = tokenizer.tokenize(text)    
    for w in words:
        if w in gli_dict:
            gli_dict[w].append(recordnum)
        else:
            gli_dict[w] = [recordnum]

In [62]:
glig_df = pd.DataFrame({'WORDS': gli_dict.keys(), 'RECORDS': gli_dict.values()})

glig_df.to_csv(results_path+config_dict['ESCREVA'][0], index=False, sep=';')